# **지능형 IoT 응용** (2021년 2학기)
#### 한국기술교육대학교 컴퓨터공학부 스마트 IoT 트랙


---

# 05. TensorFlow Lite for Microcontrollers를 이용한 애플리케이션 구축 및 배포

---

### Acknowledgement


이 자료는 다음 서적의 내용을 바탕으로 작성되었음
- 초소형 머신러닝 TinyML (5, 6장), 피트 워든, 대니얼 시투나야케 지음, 맹윤호, 임지순 옮김, 한빛미디어

---

## TinyML 'Hello World' 애플리케이션 구축

- 이전 내용에서 sin 함수 모델을 생성하는 코드를 살펴보고 모델을 생성하는 것을 직접 해보았음
- 그러나 모델은 머신러닝 애플리케이션의 일부이며, 모델 자체는 스스로 아무것도 할 수 없는 정보 덩어리임
- 모델을 사용하기 위해서는 모델을 실행하는 데 필요한 환경을 설정하고 모델에 입력을 제공하고, 모델의 출력을 사용하여 어떤 동작을 수행하는 애플리케이션 코드가 필요함
- 여기서는 이전에 구축한 사인 모델을 사용하여 간단히 LED를 점멸하게 만드는 임베디드 애플리케이션을 구현하는 것을 살펴볼 것임
 - x 값을 모델에 공급
 - 모델 추론 실행
 - 모델의 출력 결과를 사용하여 LED 점멸
 - 위 과정을 연속적으로 수행하는 애플리케이션


#### 기본적인 TinyML 애플리케이션의 아키텍처

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fb2xWLR%2Fbtq6Vrj1ehU%2FwvuIUEas0EGcJkeyMhET5k%2Fimg.png">
<center> (이미지 출처: https://toitoitoi79.tistory.com/99) </center>

- 완성된 코드는 TFLM github 저장소에서 확인할 수 있음
 - 복잡한 로직을 피하면서 TinyML 애플리케이션의 전체 구현을 최소화하도록 설계된 C++ 11 프로그램 
 - 예제 코드 경로: https://github.com/tensorflow/tflite-micro/tree/main/tensorflow/lite/micro/examples/hello_world
 - 이 예제 코드를 이해하면 TFLM 프로그램의 일반적인 구조를 이해하고 자신의 프로젝트에 동일한 구조를 재사용할 수 있게 될 것임 


---

### Notice
- __현재 TFLM 저장소의 코드는 계속 업데이트 되고 있기 때문에 책의 코드와 다른 부분도 있음을 참고__
 - TFLM github 저장소 - https://github.com/tensorflow/tflite-micro

- __C++ 코드는 github 저장소 링크에서 확인__

- __아래의 실행 가능 코드는 Colab에서 실행__

---



## 5.1 테스트 작성

- 테스트 코드 
 - 모델을 로드하고 추론을 실행해 원하는 예측이 이루어지는지 확인하는 테스트 코드를 먼저 확인할 것임
 - 테스트 코드를 실행해봄으로써 구현하려는 애플리케이션 코드가 예상대로 작동한다는 것을 먼저 검증해볼 수 있음
 - hello_world_test.cc 파일: https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/examples/hello_world/hello_world_test.cc
 - 애플리케이션의 핵심적인 코드를 파악할 수 있음

### 5.1.1 종속성 불러오기

```cpp
#include <math.h>

#include "tensorflow/lite/micro/all_ops_resolver.h"
#include "tensorflow/lite/micro/examples/hello_world/hello_world_model_data.h"
#include "tensorflow/lite/micro/micro_error_reporter.h"
#include "tensorflow/lite/micro/micro_interpreter.h"
#include "tensorflow/lite/micro/testing/micro_test.h"
#include "tensorflow/lite/schema/schema_generated.h"
```


- tensorflow/lite/micro/all_ops_resolver.h
 - TFLM 인터프리터(모델 실행하는 역할)가 모델에서 사용하는 TensorFlow operation(텐서플로우에서 계산을 수행하는 핵심 노드)을 로드할 수 있게 하는 클래스
 - https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/all_ops_resolver.h

- tensorflow/lite/micro/examples/hello_world/hello_world_model_data.h
 - C++ 코드로 변환한 사인 함수 모델 데이터 파일
 - 현재 example/hello_world 경로 상에는 이 헤더 파일은 없고 hello_world.tflite 파일이 있음. 컴파일 과정에서 이 헤더 파일과 C++ 파일이 해당 경로로 생성되게 되어 있음
 - https://github.com/tensorflow/tflite-micro/tree/main/tensorflow/lite/micro/examples/hello_world

- tensorflow/lite/micro/micro_error_reporter.h
 - 디버깅을 위해 오류와 출력을 로깅하는 클래스
 - https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/micro_error_reporter.h

- tensorflow/lite/micro/micro_interpreter.h
 - 모델을 실행하는 마이크로컨트롤러용 텐서플로우 라이트 인터프리터 클래스
 - https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/micro_interpreter.h

- tensorflow/lite/micro/testing/micro_test.h
 - 테스트 작성을 위한 간단한 프레임워크. 테스트를 위한 여러 가지 매크로가 정의되어 있음
 - 매크로: 코드 덩어리에 이름을 붙여서 다른 곳에 포함하여 재사용 가능하도록 정의한 것
 - https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/testing/micro_test.h

- tensorflow/lite/schema/schema_generated.h
 - TensorFlow Lite FlatBuffer 데이터 구조를 정의하는 스키마
 - https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/schema/schema_generated.h

### 5.1.2 테스트 설정

```cpp
TF_LITE_MICRO_TESTS_BEGIN

TF_LITE_MICRO_TEST(LoadModelAndPerformInference) {
```

- TF_LITE_MICRO_TESTS_BEGIN, TF_LITE_MICRO_TEST 
 - micro_test.h 헤더 파일에 정의된 매크로 이름
 - https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/testing/micro_test.h


```cpp
  // Define the input and the expected output
  float x = 0.0f;
  float y_true = sin(x);
```
- x 값과 이에 대한 실제 사인 함수 값(정답치) 변수 정의

### 5.1.3 데이터 기록 준비

```cpp
  // Set up logging
  tflite::MicroErrorReporter micro_error_reporter;
```

- MicroErrorReporter 인스턴스 정의
 - micro_error_reporter.h에 정의되어 있으며 실행 중 디버그 정보를 기록하는 기능 제공
 - https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/micro_error_reporter.h

### 5.1.4 모델 매핑

```cpp
  // Map the model into a usable data structure. This doesn't involve any
  // copying or parsing, it's a very lightweight operation.
  const tflite::Model* model = ::tflite::GetModel(g_hello_world_model_data);
  if (model->version() != TFLITE_SCHEMA_VERSION) {
    TF_LITE_REPORT_ERROR(&micro_error_reporter,
                         "Model provided is schema version %d not equal "
                         "to supported version %d.\n",
                         model->version(), TFLITE_SCHEMA_VERSION);
  }
```

- Model 구조체의 포인터 변수(model) 생성
 - Model 구조체, GetModel() 메소드는 schema_generated.h에 정의됨
 - GetModel() 메소드는 Model 타입의 포인터 반환
 - 모델 데이터를 담고 있음
 - g_hello_world_model_data: 모델 데이터 배열

- 모델 버전 번호 확인
 - 모델 버전 번호를 검색하는 메소드(model->version())을 호출하여 현재 사용 중인 텐서플로우 라이트 라이브러리의 버전을 담은 TFLITE_SCHEMA_VERSION과 모델의 버전 번호를 비교
 - 숫자가 일치하면 모델이 호환되는 버전의 텐서플로우 라이트 변환기로 변환됐다는 의미

- 버전 번호가 일치하지 않으면 매크로를 통해 에러를 기록함
 - TF_LITE_REPORT_ERROR는 miro_error_reporter.h에서 참조하는 tensorflow/lite/core/api/error_reporter.h에 정의된 매크로
 - https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/core/api/error_reporter.h
 - error_reporter의 Report() 메소드를 호출하며 printf()와 유사하게 동작
 - 두개의 %d 형식지정자 자리에 model->version(), TFLITE_SCHEMA_VERSION 값 출력




### 5.1.5 AllOpsResolver 생성

```cpp
  // This pulls in all the operation implementations we need
  tflite::AllOpsResolver resolver;
```

- AllOpsResolver 클래스 인스턴스 생성
 - AllOpsResolver 클래스는 all_ops_resolver.h에 정의된 클래스이며 마이크로컨트롤러용 텐서플로우 라이트 인터프리터가 TFLM에서 사용 가능한 모든 연산(Op)에 접근할 수 있도록 함

### 5.1.6 텐서 아레나 정의

```cpp
  constexpr int kTensorArenaSize = 2000;
  uint8_t tensor_arena[kTensorArenaSize];
```

- 모델이 실행되는 동안 필요한 작업 메모리 영역 할당 필요
- 텐서 아레나(tensor arena): 모델의 입력, 출력, 중간 텐서를 저장하는 데 사용되는 모메리 영역
- 2000 바이트 8비트 int 배열을 할당함

- 텐서 아레나의 적절한 크기?
 - 모델 아키텍처마다 크기, 입력, 출력, 중간 텐서가 다르므로 텐서 아레나의 적절한 크기는 알기 어려움
 - 마이크로컨트롤러는 RAM이 제한되어 있으므로 가능한 한 텐서 아레나를 작게 유지하여 나머지 프로그램을 위한 공간을 확보할 필요가 있음
 - 시행착오를 통해 최적의 크기를 찾아가는 방법이 있음 : 큰 크기부터 시작해서 모델이 실행되지 않을 때까지 크기를 줄여가는 방식

### 5.1.7 인터프리터 생성

```cpp
  // Build an interpreter to run the model with
  tflite::MicroInterpreter interpreter(model, resolver, tensor_arena,
                                       kTensorArenaSize, &micro_error_reporter);
  // Allocate memory from the tensor_arena for the model's tensors
  TF_LITE_MICRO_EXPECT_EQ(interpreter.AllocateTensors(), kTfLiteOk);
```

- MicroInterpreter 클래스의 인스턴스 생성
 - TFLM에서 모델을 실행하는 기능 수행
 - Model, AllOpsResolver 객체, tensor arena 배열 등을 생성자로 전달

- AllocateTensors() 메소드 호출
 - 모델이 정의한 모든 텐서를 확인한 후 tensor_arena에서 각 텐서로 메모리를 할당함
 - 추론 실행 전 반드시 호출 필요

- TF_LITE_MICRO_EXPECT_EQ 매크로
 - micro_test.h에 정의된 매크로
 - 2개의 인수의 값이 같은지 확인
 - 같지 않을 경우 로그 메시지를 출력하고 테스트가 실패한 것으로 됨
 



### 5.1.8 입력 텐서 검사

```cpp
  // Obtain a pointer to the model's input tensor
  TfLiteTensor* input = interpreter.input(0);

  // Make sure the input has the properties we expect
  TF_LITE_MICRO_EXPECT_NE(nullptr, input);
  // The property "dims" tells us the tensor's shape. It has one element for
  // each dimension. Our input is a 2D tensor containing 1 element, so "dims"
  // should have size 2.
  TF_LITE_MICRO_EXPECT_EQ(2, input->dims->size);
  // The value of each element gives the length of the corresponding tensor.
  // We should expect two single element tensors (one is contained within the
  // other).
  TF_LITE_MICRO_EXPECT_EQ(1, input->dims->data[0]);
  TF_LITE_MICRO_EXPECT_EQ(1, input->dims->data[1]);
  // The input is an 8 bit integer value
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteInt8, input->type);
```

- 모델의 입력 텐서에 대한 포인터를 얻기 위해 인터프리터의 input() 메소드 호출
 - 메소드의 인자: 원하는 텐서를 지정하는 인덱스
 - 이 예제에서는 입력 텐서가 하나만 있으므로 인덱스는 0

- TfLiteTensor
 - TensorFlow Lite에서 사용되는 텐서 구조체
 - tensorflow/lite/c/common.h에 정의됨: https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/c/common.h
 - 아래 테스트 코드에서 이를 이용하여 텐서의 shape과 dimension, type을 확인함

- TF_LITE_MICRO_EXPECT_NE
 - TF_LITE_MICRO_EXPECT_EQ 매크로와 마찬가지로 micro_test.h에 정의된 매크로
 - 두 개 인수가 같지 않음을 확인
 - 같을 경우 로그 메시지를 출력하고 테스트가 실패한 것으로 됨

- TF_LITE_MICRO_EXPECT_NE(nullptr, input);
 - 입력 텐서(input)이 null pointer가 아닌지 확인

- TF_LITE_MICRO_EXPECT_EQ(2, input->dims->size);
 - 입력 텐서(input)의 dimension(dims) 크기가 2와 같은지 확인
 - 우리가 사용하는 모델의 입력은 스칼라 값이지만 케라스 레이어가 입력을 받아들이는 방식에 맞추려면 입력값을 하나의 숫자를 포함하는 2D 텐서로 만들어서 제공해야 함
 - 입력이 0인 경우 [[0]]이 되어야 함
 - 따라서 입력 데이터의 차원은 2가 되어야 함

- TF_LITE_MICRO_EXPECT_EQ(1, input->dims->data[0]);  TF_LITE_MICRO_EXPECT_EQ(1, input->dims->data[1]);
 - 텐서 구조가 올바른지 확인
 - dims의 data 변수는 각 차원마다 해당 차원의 크기를 나타내는 정수를 원소로 갖는 배열
 - 여기서 우리가 사용하는 입력의 형태는 각 차원에 하나의 원소를 포함하는 2차원 텐서이므로 dims의 data 배열의 0, 1번 원소는 모두 1이 되어야 함

- TF_LITE_MICRO_EXPECT_EQ(kTfLiteInt8, input->type);
 - 입력 텐서의 type이 올바른지 확인
 - 여기서는 8비트 integer(정수)임을 확인함


### 5.1.9 입력 데이터 양자화 및 추론 실행

```cpp
  // Get the input quantization parameters
  float input_scale = input->params.scale;
  int input_zero_point = input->params.zero_point;

  // Quantize the input from floating-point to integer
  int8_t x_quantized = x / input_scale + input_zero_point;
  // Place the quantized input in the model's input tensor
  input->data.int8[0] = x_quantized;
```

- 입력 데이터 양자화(quantization)
 - 입력 데이터로 8비트 정수 형을 사용하므로 부동소수점(float) 형 변수 x 값을 변환(x_quantized)
 - 양자화를 위한 파라미터는 input->params.scale, input->params.zero_point 변수 값 사용
 - 양자화 수행: int8_t x_quantized = x / input_scale + input_zero_point;
 - 모델의 입력 텐서 데이터로 양자화된 입력 변수 값 할당: input->data.int8[0] = x_quantized;
 - input->data: TfLiteTensor 구조체의 data 변수. 이는 TfLitePtrUnion 자료형으로 동일한 위치의 메모리에 서로 다른 자료형의 데이터를 저장할 수 있는 특수한 C++ 자료형임


```cpp
  // Run the model and check that it succeeds
  TfLiteStatus invoke_status = interpreter.Invoke();
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteOk, invoke_status);
```

- 인터프리터로 모델 실행
 - TfLiteStatus invoke_status = interpreter.Invoke();
 - Invoke() 메소드를 호출하면 모델이 실행되며, 입력 데이터를 이용하여 모델의 결과를 생성함
 - 모델 실행 결과는 모델의 출력 텐서에 저장됨
 - Invoke() 메소드는 TfLiteStatus 객체를 반환하여 실행의 성공 여부를 알려줌
 - 반환값은 kTfLiteOk 또는 kTfLiteError 중 하나
 - TF_LITE_MICRO_EXPECT_EQ 매크로를 통해 반환값이 kTfLiteOk인지 확인




### 5.1.10 출력 확인

```cpp
  // Obtain a pointer to the output tensor and make sure it has the
  // properties we expect. It should be the same as the input tensor.
  TfLiteTensor* output = interpreter.output(0);
  TF_LITE_MICRO_EXPECT_EQ(2, output->dims->size);
  TF_LITE_MICRO_EXPECT_EQ(1, output->dims->data[0]);
  TF_LITE_MICRO_EXPECT_EQ(1, output->dims->data[1]);
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteInt8, output->type);
```

- 모델의 출력 텐서에 대한 포인터를 얻기 위해 interpreter의 output() 메소드 호출
 - 출력은 입력과 마찬가지로 2D 텐서 안에 포함된 8비트 정수 스칼라 값

- 테스트 프레임워크의 매크로를 이용해 출력 텐서의 차원, 크기, 자료형이 올바른지 확인


```cpp
  // Get the output quantization parameters
  float output_scale = output->params.scale;
  int output_zero_point = output->params.zero_point;

  // Obtain the quantized output from model's output tensor
  int8_t y_pred_quantized = output->data.int8[0];
  // Dequantize the output from integer to floating-point
  float y_pred = (y_pred_quantized - output_zero_point) * output_scale;
```

- Dequantization 수행
 - 8비트 정수 출력 데이터를 부동소수점 데이터로 변환
 - 부동소수점 입력을 양자화하여 8비트 정수 데이터를 모델의 입력으로 사용한 것과 반대 연산


```cpp
  // Check if the output is within a small range of the expected output
  float epsilon = 0.05f;
  TF_LITE_MICRO_EXPECT_NEAR(y_true, y_pred, epsilon);
```

- 모델의 출력(y_pred)이 정답치(y_true)와 비슷한지 확인
 - TF_LITE_MICRO_EXPECT_NEAR 매크로
 - 첫 번째 인수(y_true)와 두 번째 인수(y_pred)의 차이가 세 번째 인수(epsilon)의 값(0.05f)보다 작은지 확인


```cpp
  // Run inference on several more values and confirm the expected outputs
  x = 1.f;
  y_true = sin(x);
  input->data.int8[0] = x / input_scale + input_zero_point;
  interpreter.Invoke();
  y_pred = (output->data.int8[0] - output_zero_point) * output_scale;
  TF_LITE_MICRO_EXPECT_NEAR(y_true, y_pred, epsilon);

  x = 3.f;
  y_true = sin(x);
  input->data.int8[0] = x / input_scale + input_zero_point;
  interpreter.Invoke();
  y_pred = (output->data.int8[0] - output_zero_point) * output_scale;
  TF_LITE_MICRO_EXPECT_NEAR(y_true, y_pred, epsilon);

  x = 5.f;
  y_true = sin(x);
  input->data.int8[0] = x / input_scale + input_zero_point;
  interpreter.Invoke();
  y_pred = (output->data.int8[0] - output_zero_point) * output_scale;
  TF_LITE_MICRO_EXPECT_NEAR(y_true, y_pred, epsilon);
```

- 이전의 테스트와 다른 입력 값에 대해 모델이 잘 작동하는지 추가로 검증하기 위해 추가적인 테스트 수행


### 5.1.11 테스트 실행

- 원래 목적은 마이크로컨트롤러에서 프로그램을 실행하는 것이지만 개발 시스템에서 코드를 빌드하고 실행할 수도 있음. 이를 통해 코드 작성과 디버그가 훨씬 쉬워짐
- 개인용 컴퓨터는 마이크로컨트롤러와 비교하여 출력을 로깅하고 코드를 검토하는데 편리한 도구를 제공하므로 버그를 훨씬 쉽게 파악할 수 있음
- 마이크로컨트롤러 기기에 코드를 배포하고 실행하는 데에도 시간이 걸리므로 개발 시스템 환경에서 코드를 실행해 보는 것이 훨씬 빠름


---
#### Notice
- 프로그램 빌드를 위해 Make를 사용하는데 Windows 환경에서는 사용하기에 어려움이 있음
- 아래 코드는 Colab에서 실행됨
- 리눅스 혹은 맥 OS 기반의 컴퓨터에서 코드를 직접 다운 받아서 로컬에서 실행해볼 수 있을 것임



#### TFLM 소스 코드 다운받기

- 기존 TensorFlow 저장소에서 TFLM 용 저장소로 이동되어 github 주소가 책과 달라짐 (As of June 25, 2021, the TFLM codebase has moved to a stand-alone github repository.)

- git clone 명령어를 이용하여 저장소 코드를 복제함

- /content 디렉토리는 Colab 실행시 홈 디렉토리
 - Colab 실행시 기본으로 생성되어 있는 sample_data 디렉토리와 같은 경로 상에 tflite-micro 디렉토리가 생성됨

In [ ]:
% cd /content
% rm -rf tensorflow
! git clone https://github.com/tensorflow/tflite-micro


/content
Cloning into 'tflite-micro'...
remote: Enumerating objects: 3751, done.
remote: Counting objects: 100% (933/933), done.
remote: Compressing objects: 100% (367/367), done.
remote: Total 3751 (delta 730), reused 661 (delta 561), pack-reused 2818
Receiving objects: 100% (3751/3751), 6.41 MiB | 15.45 MiB/s, done.
Resolving deltas: 100% (2442/2442), done.


#### 테스트 프로그램 빌드

- Make 
 - 소프트웨어 빌드 작업을 자동화하는 도구
 - 1976년부터 사용됨
 - 개발자는 Makefile이라는 빌드 파일을 작성하여 Make 에 코드를 빌드하고 실행하는 방법을 지시함
 - TFLM 용 Makefile은 micro/tools/make/Makefile에 정의되어 있음

- make 명령어 
 - 사용한 Makefile 경로 지정
 - 빌드하려는 구성 요소인 target 지정: 빌드하고자 하는 대상은 hello_world_test임. 여기에 접두어 test_ 를 붙여줌

- 아래 명령러를 실행하면 필요한 라이브러리와 도구가 다운되고 테스트 파일과 필요한 라이브러리를 빌드하여 실행 파일을 생성
- 빌드가 완료되면 프로그램을 실행하여 결과가 출력됨
 - 아래와 같이 출력되면 테스트 프로그램이 실행되고 모든 테스트가 성공했음을 나타냄

```
Testing LoadModelAndPerformInference
1/1 tests passed
~~~ALL TESTS PASSED~~~
``` 

In [ ]:
% cd /content/tflite-micro/

! make -f tensorflow/lite/micro/tools/make/Makefile test_hello_world_test

/content/tflite-micro
find: ‘../google/’: No such file or directory
--2021-08-16 08:41:17--  https://github.com/google/flatbuffers/archive/dca12522a9f9e37f126ab925fd385c807ab4f84e.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/google/flatbuffers/zip/dca12522a9f9e37f126ab925fd385c807ab4f84e [following]
--2021-08-16 08:41:17--  https://codeload.github.com/google/flatbuffers/zip/dca12522a9f9e37f126ab925fd385c807ab4f84e
Resolving codeload.github.com (codeload.github.com)... 192.30.255.120
Connecting to codeload.github.com (codeload.github.com)|192.30.255.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/tmp/tmp.920GinUj5W/dca12522a9f9e37f126ab925fd385c807ab4f84e.zip’

/tmp/tmp.920GinUj5W     [  <=>               ]   1.68M  6.78MB/s    in 0.2s    

2021-0

#### 테스트 프로그램 실행하기

- 아래 경로에서 빌드된 테스트 프로그램을 실행할 수 있음

In [ ]:
%cd /content/tflite-micro/

! tensorflow/lite/micro/tools/make/gen/linux_x86_64_default/bin/hello_world_test

/content/tflite-micro
Testing LoadModelAndPerformInference
1/1 tests passed
~~~ALL TESTS PASSED~~~



## 5.2 프로젝트 파일 구조

- 애플리케이션 루트
 - tensorflow/lite/micro/examples/hello_world/
 - https://github.com/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/examples/hello_world

- 주요 파일
 - Makefile.inc: 테스트 코드인 hello_world_test와 기본 애플케이션 코드인 hello_world를 포함하여 애플리케이션 내의 빌드 대상에 대한 정보가 포함된 Makefile

 - constants.h, constants.cc: 프로그램에서 사용되는 주요 상수를 정의

 - hello_world.tflite: 모델 데이터를 담고 있는 TensorFlow Lite 모델 파일. 이 파일을 이용하여 빌드 과정에서 hello_world_model_data.h,hello_world_model_data.cc 파일이 생성됨 

 - hello_world_test.cc: 위에서 살펴본 테스트 프로그램

 - main.cc: 애플리케이션이 장치에 배포되면 가장 먼저 실행되는 프로그램 진입점

 - main_functions.h, main_functions.cc: 프로그램 실행에 필요한 초기화를 수행하는 setup() 함수와 프로그램의 핵심 로직을 반복하여 실행하는 loop() 함수를 정의하는 파일. main.cc에서 이 함수들을 호출함

 - output_handler.h, output_handler.cc: 모델에 의한 추론이 실행될 때마다 모델의 출력을 표시하는데 사용되는 함수를 정의하는 파일 쌍. output_handler.cc의 기본 구현은 결과를 화면에 출력함. 이 구현을 재정의하면 다른 장치에서 다르게 동작하도록 할 수 있음

 - output_handler_test.c: output_handler.h/cc 코드가 올바른지 확인하는 테스트 프로그램

- arduino/ 디렉토리
 - 아두이노에서 동작하도록 프로그램이 빌드되는 경우 사용되는 소스 파일의 커스텀 버전 파일 포함 

 - main.cc, constants.cc, output_hanlder.cc 파일이 있음

 - output_handler.cc는 아두이노에서 프로그램 실행 시 결과를 화면에 출력하는 것이 아니라 LED를 점멸하도록 제어하는 코드를 포함

## 5.3 애플리케이션 소스 코드



### 5.3.1 main.cc

- main() 함수
 - C++ 프로그램에서 프로그램의 시작이 되는 전역 함수
 - main.cc 파일에 정의되어 있음
 - 마이크로컨트롤러가 시작될 때마다 실행됨
 - main_functions.h에 정의된 setup(), loop() 함수를 이용

 ```cpp
 #include "tensorflow/lite/micro/examples/hello_world/main_functions.h"
// This is the default main used on systems that have the standard C entry
// point. Other devices (for example FreeRTOS or ESP32) that have different
// requirements for entry code (like an app_main function) should specialize
// this main.cc file in a target-specific subfolder.
int main(int argc, char* argv[]) {
    setup();
    while (true) {
      loop();
    }
}
```


### 5.3.2 main_functions.cc


#### 코드 주요 기능

- 전역 변수 초기화

- 모델 실행을 위한 준비: setup() 함수
 - Model 객체 생성
 - AllOpsResolver 객체 생성
 - Interpreter 객체 생성
 - 텐서 메모리 할당

- 모델 실행 및 결과 처리: loop() 함수
 - 모델 입력 준비
 - Interpreter 실행
 - 모델 출력 처리


#### main_functions.cc 코드 설명

- 필요한 헤더 파일 선언
 - 위에서 살펴본 테스트 프로그램에서 사용한 것과 유사

 ```cpp
#include "tensorflow/lite/micro/examples/hello_world/main_functions.h"
#include "tensorflow/lite/micro/all_ops_resolver.h"
#include "tensorflow/lite/micro/examples/hello_world/constants.h"
#include "tensorflow/lite/micro/examples/hello_world/hello_world_model_data.h"
#include "tensorflow/lite/micro/examples/hello_world/output_handler.h"
#include "tensorflow/lite/micro/micro_error_reporter.h"
#include "tensorflow/lite/micro/micro_interpreter.h"
#include "tensorflow/lite/micro/system_setup.h"
#include "tensorflow/lite/schema/schema_generated.h"
```


- 주요 변수 초기화
 - 위에서 살펴본 테스트 프로그램에서 사용한 것과 유사
 
 - namespace로 묶여 있어서 main_functions.cc의 어느 곳에서나 접근할 수 있지만 다른 파일에서는 접근할 수 없음. 두 개의 서로 다른 파일이 동일한 이름의 변수를 정의할 때 생기는 문제를 방지할 수 있음

 - inference_count 변수: 추론 횟수를 저장하는 변수


```cpp
// Globals, used for compatibility with Arduino-style sketches.
namespace {
tflite::ErrorReporter* error_reporter = nullptr;
const tflite::Model* model = nullptr;
tflite::MicroInterpreter* interpreter = nullptr;
TfLiteTensor* input = nullptr;
TfLiteTensor* output = nullptr;
int inference_count = 0;

constexpr int kTensorArenaSize = 2000;
uint8_t tensor_arena[kTensorArenaSize];
}  // namespace
```


- setup() 함수 
 - 디버그용 메시지 로깅을 위한 ErrorReporter 객체 생성
 - 모델 객체 생성
 - 모델에서 사용할 Op 접근을 위한 AllOpsResolver 객체 생성
 - 모델 실행을 위한 MicroInterpreter 객체 생성
 - 모델에서 사용될 텐서를 tensor arena의 메모리에 할당
 - 입력, 출력 텐서에 대한 포인터 가져옴
 - 추론 횟수를 저장할 변수 0으로 설정


```cpp
// The name of this function is important for Arduino compatibility.
void setup() {
  tflite::InitializeTarget();

  // Set up logging. Google style is to avoid globals or statics because of
  // lifetime uncertainty, but since this has a trivial destructor it's okay.
  // NOLINTNEXTLINE(runtime-global-variables)
  static tflite::MicroErrorReporter micro_error_reporter;
  error_reporter = &micro_error_reporter;

  // Map the model into a usable data structure. This doesn't involve any
  // copying or parsing, it's a very lightweight operation.
  model = tflite::GetModel(g_hello_world_model_data);
  if (model->version() != TFLITE_SCHEMA_VERSION) {
    TF_LITE_REPORT_ERROR(error_reporter,
                         "Model provided is schema version %d not equal "
                         "to supported version %d.",
                         model->version(), TFLITE_SCHEMA_VERSION);
    return;
  }

  // This pulls in all the operation implementations we need.
  // NOLINTNEXTLINE(runtime-global-variables)
  static tflite::AllOpsResolver resolver;

  // Build an interpreter to run the model with.
  static tflite::MicroInterpreter static_interpreter(
      model, resolver, tensor_arena, kTensorArenaSize, error_reporter);
  interpreter = &static_interpreter;

  // Allocate memory from the tensor_arena for the model's tensors.
  TfLiteStatus allocate_status = interpreter->AllocateTensors();
  if (allocate_status != kTfLiteOk) {
    TF_LITE_REPORT_ERROR(error_reporter, "AllocateTensors() failed");
    return;
  }

  // Obtain pointers to the model's input and output tensors.
  input = interpreter->input(0);
  output = interpreter->output(0);

  // Keep track of how many inferences we have performed.
  inference_count = 0;
}
```


- loop() 함수
 - 모델 입력으로 사용할 x 값 계산: kXrange, kInferencesPerCycle은 constants.h에 정의된 상수. kXrange는 x의 가능한 최대값을 2파이로 정의. kInferencesPerCycle은 x 값이 0-2파이까지 변하는 한 사이클 동안 수행할 추론 횟수 정의
 - 부동소수점으로 계산된 x 값을 8비트 정수로 양자화 수행
 - 추론 실행: MicroInterpreter의 Invoke() 메소드 호출
 - 모델 출력 값(8비트 정수)을 부동소수점 데이터로 변환
 - 출력 처리: HandleOutput() 함수 호출
 - inference_count 변수 업데이트


```cpp
// The name of this function is important for Arduino compatibility.
void loop() {
  // Calculate an x value to feed into the model. We compare the current
  // inference_count to the number of inferences per cycle to determine
  // our position within the range of possible x values the model was
  // trained on, and use this to calculate a value.
  float position = static_cast<float>(inference_count) /
                   static_cast<float>(kInferencesPerCycle);
  float x = position * kXrange;

  // Quantize the input from floating-point to integer
  int8_t x_quantized = x / input->params.scale + input->params.zero_point;
  // Place the quantized input in the model's input tensor
  input->data.int8[0] = x_quantized;

  // Run inference, and report any error
  TfLiteStatus invoke_status = interpreter->Invoke();
  if (invoke_status != kTfLiteOk) {
    TF_LITE_REPORT_ERROR(error_reporter, "Invoke failed on x: %f\n",
                         static_cast<double>(x));
    return;
  }

  // Obtain the quantized output from model's output tensor
  int8_t y_quantized = output->data.int8[0];
  // Dequantize the output from integer to floating-point
  float y = (y_quantized - output->params.zero_point) * output->params.scale;

  // Output the results. A custom HandleOutput function can be implemented
  // for each supported hardware target.
  HandleOutput(error_reporter, x, y);

  // Increment the inference_counter, and reset it if we have reached
  // the total number per cycle
  inference_count += 1;
  if (inference_count >= kInferencesPerCycle) inference_count = 0;
}
```



### 5.3.3 output_handler.cc

- 주요 기능
 - ErrorReporter 인스턴스를 사용하여 x, y 값을 기록
 - 컴퓨터에서 실행 시 화면에 값이 출력됨


```cpp
#include "tensorflow/lite/micro/examples/hello_world/output_handler.h"

void HandleOutput(tflite::ErrorReporter* error_reporter, float x_value, float y_value) {
  // Log the current X and Y values
  TF_LITE_REPORT_ERROR(error_reporter, "x_value: %f, y_value: %f\n",
                       static_cast<double>(x_value),
                       static_cast<double>(y_value));
}
```

- 아두이노에서 프로그램이 실행되는 경우 arduino 디렉토리의 output_handler.cc 파일로 대체됨

### 5.3.5 애플리케이션 실행하기

#### Hello World 예제 빌드

- 이전에 살펴본 테스트 프로그램과 마찬가지로 make 명령어를 이용하여 빌드
 - TFLM 코드가 다운로드된 상태이어야 함 (git clone 사용)

In [ ]:
% cd /content/tflite-micro/

! make -f tensorflow/lite/micro/tools/make/Makefile hello_world

/content/tflite-micro
find: ‘../google/’: No such file or directory
tensorflow/lite/micro/tools/make/downloads/flatbuffers already exists, skipping the download.
tensorflow/lite/micro/tools/make/downloads/pigweed already exists, skipping the download.
tensorflow/lite/micro/tools/make/downloads/person_model_int8 already exists, skipping the download.
g++ -std=c++11 -fno-rtti -fno-exceptions -fno-threadsafe-statics -Werror -fno-unwind-tables -ffunction-sections -fdata-sections -fmessage-length=0 -DTF_LITE_STATIC_MEMORY -DTF_LITE_DISABLE_X86_NEON -Wsign-compare -Wdouble-promotion -Wshadow -Wunused-variable -Wmissing-field-initializers -Wunused-function -Wswitch -Wvla -Wall -Wextra -Wstrict-aliasing -Wno-unused-parameter -DLINUX -DTF_LITE_USE_CTIME -Os -I. -Itensorflow/lite/micro/tools/make/downloads/gemmlowp -Itensorflow/lite/micro/tools/make/downloads/flatbuffers/include -Itensorflow/lite/micro/tools/make/downloads/ruy -Itensorflow/lite/micro/tools/make/gen/linux_x86_64_default/genfiles/

#### 애플리케이션 실행

- 프로그램은 입력 x에 대한 sin(x) 예측값 y를 계속 출력함

 - output_handler.cc의 HandleOutput() 함수로 작성된 로그
 - 한 번의 추론에 한 줄의 로그가 출력됨

In [ ]:
%cd /content/tflite-micro/

! tensorflow/lite/micro/tools/make/gen/linux_x86_64_default/bin/hello_world 

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
x_value: 1.2566366*2^-2, y_value: 1.4910722*2^-2

x_value: 1.2566366*2^-1, y_value: 1.1183041*2^-1

x_value: 1.8849551*2^-1, y_value: 1.6774564*2^-1

x_value: 1.2566366*2^0, y_value: 1.9316164*2^-1

x_value: 1.5707957*2^0, y_value: 1.0420563*2^0

x_value: 1.8849551*2^0, y_value: 1.9146728*2^-1

x_value: 1.0995567*2^1, y_value: 1.6435688*2^-1

x_value: 1.2566366*2^1, y_value: 1.0674724*2^-1

x_value: 1.4137159*2^1, y_value: 1.8977287*2^-3

x_value: 1.5707957*2^1, y_value: 1.0844163*2^-7

x_value: 1.7278753*2^1, y_value: -1.2199684*2^-2

x_value: 1.8849551*2^1, y_value: -1.0674724*2^-1

x_value: 1.0210171*2^2, y_value: -1.5588485*2^-1

x_value: 1.0995567*2^2, y_value: -1.9316164*2^-1

x_value: 1.1780966*2^2, y_value: -1.1098324*2^0

x_value: 1.2566366*2^2, y_value: -1.9655047*2^-1

x_value: 1.3351763*2^2, y_value: -1.4910722*2^-1

x_value: 1.4137159*2^2, y_value: -1.0674724*2^-1

x_value: 1.4922558*2^2, y_value: -1.4232964*2^-2

x_value: 1.0*2^-127, y_

---
## 마이크로컨트롤러에 애플리케이션 배포하기

---

## 5.4 개발 환경

### Arduino IDE 이용
Arduino IDE
- Arduino IDE 최신 버전 설치
- https://www.arduino.cc/en/software  

- **Arduino_TensorFlowLite 라이브러리 설치**
  - ML models을 Arduino Nano 33 BLE Sense 보드에서 구동할 수 있도록 하는 라이브러리
  - Arduino IDE 실행 후 툴 메뉴 클릭 >> 라이브러리 관리... 클릭 >> 라이브러리 매니저 창에서 Arduino_TensorFlowLite 검색 >> 2.4.0-ALPHA 버전 선택 후 설치 (2.1.0-ALPHA-precompiled)
  - 설치된 라이브러리 폴더 위치: [사용자 문서 폴더]/Arduino/libraries

- **Arduino Nano 33 BLE Sense 보드 설치**
 - Arduino IDE 실행 후 툴 메뉴 클릭 >> 보드 선택 >> 보드 매니저... 클릭 >> 검색 창에서 nano 33 검색 >> Arduino Mbed OS Nano Boards 버전 선택 후 설치 (2.2.0)



### VSCode에서 PlatformIO 이용

Visual Studio Code
 - Visual Studio Code 최신 버전 설치
 - https://code.visualstudio.com/


PlatformIO
- 임베디드 시스템 개발을 위한 Visual Studio Code (VSCode)의 extension 
- https://platformio.org/ 
- 설치를 위해서 VSCode 실행 후 윈도우 좌측의 사이드바에서 Extensions 탭 클릭 >> 검색 창에 PlatformIO IDE 검색 >> PlatformIO IDE 선택 >> extension 설명 패널에서 Install 버튼 클릭 
- PlatformIO에서 아두이노 프로젝트 import 하기, 빌드하기, 아두이노에 업로드하기
 - https://github.com/Harvard-CS249R-Fall2020/assignments/tree/master/hello_world


## 5.5 마이크로컨트롤러란 무엇인가

- Microcontroller 또는 MCU (microcontroller unit)
 - 집적 회로 안에 마이크로프로세서와 입출력 모듈, 메모리 등의 컴퓨팅 요소를 내장한 초소형 컴퓨팅 장치
 - 보통 MCU는 임베디드 애플리케이션 용으로 디자인된 연산 장치이며 임베디드 시스템에 널리 사용됨

 - [SparkFun Edge Development Board](https://www.sparkfun.com/products/15170)

 - [Arduino Nano 33 BLE Sense](https://store.arduino.cc/usa/nano-33-ble-sense)

## 5.6 아두이노

- 아두이노 보드의 종류는 매우 다양하며 그 성능도 제각각임
- 모든 아두이노가 마이크로컨트롤러용 텐서플로우 라이트를 실행할 수 있는 것은 아님
- 우리가 사용할 아두이노 보드는 Arduino Nano 33 BLE Sense
 - 텐서플로우 라이트와 호환되는 것 외에도 마이크와 가속도계가 포함되어 있음
 - 아두이노 보드에는 LED가 내장되어 있으며 LED는 사인 값을 시각적으로 출력하는데 사용됨

### 5.6.1 아두이노 출력 다루기

- 모델의 출력 결과에 따라 아두이노의 내장 LED를 제어
 - 입력 x는 0에서 2파이까지 변하므로 모델의 출력 y는 대략적으로 -1에서 1사이 값이 될 것임
 - -1인 경우 밝기를 최저로 1인 경우 밝기를 최대로 하도록 구현됨

- arduino 디렉토리의 output_handler.cc 파일의 HandleOutput() 함수에서 처리
 - brightness 변수 값을 계산하고 이를 analogWrite() 함수에서 사용함
 - 아두이노 내장 LED 핀에서 PWM을 지원하지 않으면 단순 on/off 만 가능
 - ErrorReporter를 통해서 시리얼 포트를 이용해 데이터를 기록할 수 있음. 이를 Arduino IDE 툴 메뉴의 시리얼 모니터 혹은 시리얼 플로터를 이용하여 확인할 수 있음
 - 시리얼 통신: 마이크로컨트롤러가 호스트 컴퓨터와 통신하는 일반적인 방법이며 디버깅에 흔히 사용됨
 - 시리얼 모니터나 시리얼 플로터는 시리얼 포트를 통해 수신된 데이터를 캡처하고 표시하기 위한 도구

```cpp
#include "tensorflow/lite/micro/examples/hello_world/output_handler.h"

#include "Arduino.h"
#include "tensorflow/lite/micro/examples/hello_world/constants.h"

// The pin of the Arduino's built-in LED
int led = LED_BUILTIN;

// Track whether the function has run at least once
bool initialized = false;

// Animates a dot across the screen to represent the current x and y values
void HandleOutput(tflite::ErrorReporter* error_reporter, float x_value,
                  float y_value) {
  // Do this only once
  if (!initialized) {
    // Set the LED pin to output
    pinMode(led, OUTPUT);
    initialized = true;
  }

  // Calculate the brightness of the LED such that y=-1 is fully off
  // and y=1 is fully on. The LED's brightness can range from 0-255.
  int brightness = (int)(127.5f * (y_value + 1));

  // Set the brightness of the LED. If the specified pin does not support PWM,
  // this will result in the LED being on when y > 127, off otherwise.
  analogWrite(led, brightness);

  // Log the current brightness value for display in the Arduino plotter
  TF_LITE_REPORT_ERROR(error_reporter, "%d\n", brightness);
}
```





### 5.6.2 예제 실행하기

- 아두이노용 프로젝트를 빌드하고 장치에 배포하기
- 필요한 것
 - 아두이노 나노 33 BLE 센스
 - USB 케이블
 - Arduino IDE 혹은 PlatformIO가 설치된 VS Code

#### **실행 절차**
- Arduino_TensorFlowLite 라이브러리 설치
 - Arduino IDE에 Arduino_TensorFlowLite 라이브러리가 설치되어 있어야 함
 - 필요한 예제 코드를 포함하고 있음
- 예제 로드
 - 파일 메뉴 >> 예제 >> Arduino_TensorFlowLite >> hello_world 선택
- USB 케이블로 아두이노를 컴퓨터에 연결
- Arduino IDE에서 해당 보드(Arduino Nano 33 BLE) 선택
 - 툴 메뉴 >> 보드
- Arduino IDE에서 업로드 버튼(오른쪽 방향 화살표) 클릭
 - 코드가 컴파일되고 완료되면 보드에 업로드되어 실행됨
- 업로드가 성공적으로 완료되면 LED가 점멸됨
 - 주황 LED가 주기적으로 밝기가 변함
 - 툴 메뉴 >> 시리얼 플로터 혹은 시리얼 모니터를 선택하여 시간이 지남에 따라 변화하는 값을 확인할 수 있음





#### 아두이노 예제 코드의 차이점
- Arduino IDE에서 hello_world 예제를 로드하면 관련 파일들이 열리는데 github 저장소의 코드와 약간의 차이점이 있음

- 소스 파일 확장자
 - .cc 파일은 .cpp 파일로 변경

- main_functions.cc 파일은 hello_world 파일이 됨

- Arduino IDE는 하위 디렉토리를 지원하지 않기 때문에 arduino 디렉토리에 있는 constants.cc, main.cc, output_handler.cc 파일은 파일명 앞에 디렉토리 이름이 추가되어 arduino_constants.cpp, arduino_main.cpp, arduino_output_handler.cpp 파일이 됨

- model.cpp, model.h: 모델 데이터 파일

### 5.6.3 커스텀 버전 만들기

- LED 점등 주기 변경
 - arduino_constants.cpp 파일에서 kInferencesPerCycle 상수 값을 조정하여 밝기 변화 주기를 변경할 수 있음


- 변화 속도를 더 빠르게 하거나 더 느리게 해보자

 